挂载google硬盘

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


配置kaggle环境

In [0]:

!mkdir mydata/NYC-taxi-fare
!mkdir mydata/keras-loss
!mkdir mydata/keras-weight
!cd /content
!pip install kaggle
!mkdir .kaggle
import json
token={"username":"yingjiaaa","key":"ab0c8b444dc434f83c6ba9f6b0055692"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)  
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
!kaggle config set -n path -v /content/NYC-taxi-fare/

mkdir: cannot create directory ‘mydata/NYC-taxi-fare’: File exists
mkdir: cannot create directory ‘mydata/keras-loss’: File exists
mkdir: cannot create directory ‘mydata/keras-weight’: File exists
mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: /content/NYC-taxi-fare/


下载数据并且解压缩

In [0]:
!cd /content/drive/'My Drive'/mydata/
!kaggle competitions download -c new-york-city-taxi-fare-prediction -p /content/drive/'My Drive'/mydata/
!unzip -d /content/drive/'My Drive'/mydata/ /content/drive/'My Drive'/mydata/train.csv.zip

  0% 0.00/335k [00:00<?, ?B/s]
100% 335k/335k [00:00<00:00, 22.0MB/s]
  0% 0.00/960k [00:00<?, ?B/s]
100% 960k/960k [00:00<00:00, 29.5MB/s]
100% 1.56G/1.56G [00:19<00:00, 80.6MB/s]
100% 1.56G/1.56G [00:19<00:00, 87.7MB/s]
  0% 0.00/486 [00:00<?, ?B/s]
100% 486/486 [00:00<00:00, 67.3kB/s]
Archive:  /content/drive/My Drive/mydata/train.csv.zip
  inflating: /content/drive/My Drive/mydata/train.csv  


In [0]:
!cd /content/drive/'My Drive'/mydata/

选择合适的上下车地点（根据经纬度）

In [0]:
def clean(df):
    # Delimiter lats and lons to NY only
    df = df[(-76 <= df['pickup_longitude']) & (df['pickup_longitude'] <= -72)]
    df = df[(-76 <= df['dropoff_longitude']) & (df['dropoff_longitude'] <= -72)]
    df = df[(38 <= df['pickup_latitude']) & (df['pickup_latitude'] <= 42)]
    df = df[(38 <= df['dropoff_latitude']) & (df['dropoff_latitude'] <= 42)]
    # Remove possible outliers
    df = df[(0 < df['fare_amount']) & (df['fare_amount'] <= 250)]
    # Remove inconsistent values
    df = df[(df['dropoff_longitude'] != df['pickup_longitude'])]
    df = df[(df['dropoff_latitude'] != df['pickup_latitude'])]
    return df

定义白天上车时间，夜晚上车时间，起点和终点之间的曼哈顿距离；将时间特征拆分为年、月、日、时、周、工作日并且加上白天和夜晚标记；计算起点到终点的曼哈顿距离和欧几里得距离，并且分别计算起点和终点到三个城市的曼哈顿和欧几里得距离

In [0]:
def late_night (row):
    if (row['hour'] <= 6) or (row['hour'] >= 20):
        return 1
    else:
        return 0


def night (row):
    if ((row['hour'] <= 20) and (row['hour'] >= 16)) and (row['weekday'] < 5):
        return 1
    else:
        return 0
    
    
def manhattan(pickup_lat, pickup_long, dropoff_lat, dropoff_long):
    return np.abs(dropoff_lat - pickup_lat) + np.abs(dropoff_long - pickup_long)


def add_time_features(df):
    df['pickup_datetime'] =  pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
    df['year'] = df['pickup_datetime'].apply(lambda x: x.year)
    df['month'] = df['pickup_datetime'].apply(lambda x: x.month)
    df['day'] = df['pickup_datetime'].apply(lambda x: x.day)
    df['hour'] = df['pickup_datetime'].apply(lambda x: x.hour)
    df['weekday'] = df['pickup_datetime'].apply(lambda x: x.weekday())
    df['pickup_datetime'] =  df['pickup_datetime'].apply(lambda x: str(x))
    df['night'] = df.apply (lambda x: night(x), axis=1)
    df['late_night'] = df.apply (lambda x: late_night(x), axis=1)
    # Drop 'pickup_datetime' as we won't need it anymore
    df = df.drop('pickup_datetime', axis=1)
    
    return df


def add_coordinate_features(df):
    lat1 = df['pickup_latitude']
    lat2 = df['dropoff_latitude']
    lon1 = df['pickup_longitude']
    lon2 = df['dropoff_longitude']
    
    # Add new features
    df['latdiff'] = (lat1 - lat2)
    df['londiff'] = (lon1 - lon2)
   #print("LLLLL")
    return df


def add_distances_features(df):
    # Add distances from airpot and downtown
    ny = (-74.0063889, 40.7141667)
    jfk = (-73.7822222222, 40.6441666667)
    ewr = (-74.175, 40.69)
    lgr = (-73.87, 40.77)
    
    lat1 = df['pickup_latitude']
    lat2 = df['dropoff_latitude']
    lon1 = df['pickup_longitude']
    lon2 = df['dropoff_longitude']
    df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
    df['manhattan'] = manhattan(lat1, lon1, lat2, lon2)
    df['downtown_pickup_distance'] = manhattan(ny[1], ny[0], lat1, lon1)
    df['downtown_dropoff_distance'] = manhattan(ny[1], ny[0], lat2, lon2)
    df['jfk_pickup_distance'] = manhattan(jfk[1], jfk[0], lat1, lon1)
    df['jfk_dropoff_distance'] = manhattan(jfk[1], jfk[0], lat2, lon2)
    df['ewr_pickup_distance'] = manhattan(ewr[1], ewr[0], lat1, lon1)
    df['ewr_dropoff_distance'] = manhattan(ewr[1], ewr[0], lat2, lon2)
    df['lgr_pickup_distance'] = manhattan(lgr[1], lgr[0], lat1, lon1)
    df['lgr_dropoff_distance'] = manhattan(lgr[1], lgr[0], lat2, lon2)
    
    return df

In [0]:
def output_submission(raw_test, prediction, id_column, prediction_column, file_name):
    df = pd.DataFrame(prediction, columns=[prediction_column])
    df[id_column] = raw_test[id_column]
    df[[id_column, prediction_column]].to_csv((file_name), index=False)
    print('Output complete')
import matplotlib.pyplot as plt
def plot_loss_accuracy(history,label_feature):
    plt.figure(figsize=(20,10))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(label_feature+'model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.savefig("/content/drive/My Drive/mydata/keras-loss/"+label_feature+'model_loss.jpg')
    plt.show()

In [0]:
TRAIN_PATH = "/content/drive/My Drive/mydata/train.csv"
TEST_PATH ="/content/drive/My Drive/mydata/test.csv"
SUBMISSION_NAME = 'NYC-taxi-fare/submission.csv'

# Model parameters
BATCH_SIZE = 256
EPOCHS = 50
LEARNING_RATE = 0.001
DATASET_SIZE = 6

In [0]:
!ls

drive  sample_data


In [0]:
%time
import pandas as pd
datatypes = {'key': 'str', 
              'fare_amount': 'float32',
              'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

# Only a fraction of the whole data
train = pd.read_csv(TRAIN_PATH, nrows=DATASET_SIZE)
test = pd.read_csv(TEST_PATH)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.44 µs


In [0]:
train.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

各类型的数据预处理

In [0]:
import numpy as np
a=["time","corordinate","distance"]
def get_subset(a): 
  sub=[]
  to_list=[]
  if len(a)==0:
    return []
  for i in a:
    if a.index(i)>0:
      to_list.append(i)
  sub_list=get_subset(to_list)
  if sub_list==[]:
    return([[a[0]],[]])
  #print(sub_list)
  #print("yes")
  for i in sub_list:
    #print(i)
    sub.append(i+[a[0]])
    sub.append(i)
  return sub
how_to_deal=(get_subset(a))
del(how_to_deal[7])
data_with_diff_feature={}
diff_model={}
train = clean(train)
for i in how_to_deal:
  data_deal_method=""
  train_temp=train.copy()
  test_temp=test.copy()
  #print(i)
  if  "time" in i:
    data_deal_method=data_deal_method+"_time"
    train_temp = add_time_features(train_temp)
    test_temp = add_time_features(test_temp)
  if "corordinate" in i:
    data_deal_method=data_deal_method+"_corordinate"
    add_coordinate_features(train_temp)
    add_coordinate_features(test_temp)
  if "latdiff" in train_temp and "distance" in i:
    data_deal_method=data_deal_method+"_distance"
    train_temp = add_distances_features(train_temp)
    test_temp =  add_distances_features(test_temp)
  data_with_diff_feature[data_deal_method]=[train_temp,test_temp]
  del(train_temp)
  del(test_temp) 

In [0]:
for i in data_with_diff_feature:
  print(i)

_time_corordinate_distance
_corordinate_distance
_time

_time_corordinate
_corordinate


knn回归

In [0]:
def knn(X_train,X_val,train_y,val_y,label_features):
  global metric
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.neighbors import KNeighborsRegressor
  knn_reg = KNeighborsRegressor()
  #y=pd.DataFrame(train_labels)
  #X=pd.DataFrame(train_df_scaled)
  #print(X)
  #print(pd.DataFrame(y))
  #X_train,X_val,train_y,val_y=train_test_split(X,y,random_state=0)  
  knn_reg.fit(X_train,train_y)
  from sklearn.metrics import mean_squared_error
  mse = mean_squared_error(knn_reg.predict(X_val),val_y)
  #print(mse**0.5)
  metric[label_features]=metric[label_features]+[mse**0.5]
  

初始化metric

In [0]:
metric={}
metric_1={}
del(data_with_diff_feature[""])
for i in data_with_diff_feature:
  print(i)
  metric[i]=[0]
metric["times"]=[0]
metric["data_size"]=[0]
metric["way"]=[0]
print(metric)

In [0]:

def process_evluate(train,test,label_features,data_size):
  # Drop unwanted columns
  dropped = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','pickup_datetime','key','passenger_count']
 # print("train")
  #print(train)
  #print("llabel")
 # print(label_features)
  train,test=train[:data_size],test[:data_size]
  print(train.shape)
  dropped_columns=[]
  for i in dropped:
    if i in train.columns:
      dropped_columns.append(i)
  
  train_clean = train.drop(dropped_columns, axis=1)
  test_clean = test.drop(dropped_columns + ['key', 'passenger_count'], axis=1)
  # peek data
  from sklearn.model_selection import train_test_split
  train_df, validation_df = train_test_split(train_clean, test_size=0.10, random_state=1)
  print((train_df))
  # Get labels
  train_labels = train_df['fare_amount'].values
  validation_labels = validation_df['fare_amount'].values
  train_df = train_df.drop(['fare_amount'], axis=1)
  validation_df = validation_df.drop(['fare_amount'], axis=1)
  # Scale data
  # Note: im doing this here with sklearn scaler but, on the Coursera code the scaling is done with Dataflow and Tensorflow
  %time
  #print("2")
  from sklearn import preprocessing
  #help(preprocessing.MinMaxScaler)
  scaler = preprocessing.MinMaxScaler()
 #print("0")
  #print(train_df)
  train_df_scaled = scaler.fit_transform(train_df)
  print("4")
  print(len(validation_df.columns),len(test_clean.columns))
  validation_df_scaled = scaler.transform(validation_df)
  test_scaled = scaler.transform(test_clean)
  print("3")
  knn(train_df_scaled,validation_df_scaled,train_labels,validation_labels,label_features)
  del(scaler)
  #print(train_df_scaled)
  from keras.models import Sequential
  from keras.layers import Dense, Activation, BatchNormalization
  from keras import regularizers, optimizers
  model = Sequential()
  model.add(Dense(256, activation='relu', input_dim=train_df_scaled.shape[1], activity_regularizer=regularizers.l1(0.01)))
  model.add(BatchNormalization())
  model.add(Dense(128, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(64, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(32, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(8, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1))
  adam = optimizers.adam(lr=LEARNING_RATE)
  model.compile(loss='mse', optimizer=adam, metrics=['mse'])
  history = model.fit(x=train_df_scaled, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                    verbose=1, validation_data=(validation_df_scaled, validation_labels), 
                    shuffle=True)
  y_predict=model.predict(x=validation_df_scaled)
  from sklearn.metrics import mean_squared_error
  mse = mean_squared_error(y_predict,validation_labels)
  global metric#,diff_model
  metric[label_features]=metric[label_features]+[mse**0.5]
  label_features=str(data_size)+label_features
  model.save_weights("/content/drive/My Drive/mydata/keras-weight/"+label_features+".h5")
  #diff_model[label_features]=model
  import matplotlib.pyplot as plt
  plot_loss_accuracy(history,label_features)
  #pd.DataFrame(history.history)


In [0]:
import time
data_size=[6000000]
for j in data_size:
  start=time.time()
  for i in data_with_diff_feature:
    process_evluate(data_with_diff_feature[i][0],data_with_diff_feature[i][1],i,j)
  metric["times"]=metric["times"]+[time.time()-start]
  metric["data_size"]=metric["data_size"]+[j]
  metric["times"]=metric["times"]+[time.time()-start]
  metric["data_size"]=metric["data_size"]+[j]
  metric["way"]=metric["way"]+['knn']
  metric["way"]=metric["way"]+['nero']

In [2]:
print(metric)

NameError: ignored

从文件夹下导入已经处理好的数据

In [0]:
#train_clean=pd.read_csv("/content/NYC-taxi-fare/train_clean.csv")
#test_clean=pd.read_csv("/content/NYC-taxi-fare/test_clean.csv")

In [0]:
#model.load_weights("/content/NYC-taxi-fare/net")


In [0]:
#model.save_weights("/content/NYC-taxi-fare/net.h5")
# 将模型权重保存到指定路径，文件类型是HDF5（后缀是.h5）

简单线性预测

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor()
y=pd.DataFrame(train_labels)
X=pd.DataFrame(train_df_scaled)
#print(X)
#print(pd.DataFrame(y))
X_train,X_val,train_y,val_y=train_test_split(X,y,random_state=0)  
knn_reg.fit(X_train,train_y)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(knn_reg.predict(X_val),val_y)
print(mse**0.5)

In [0]:
train_df_scaled.shape